<a href="https://colab.research.google.com/github/marcinczeczko/wtt-01-classifier-app-template/blob/main/classifier_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.vision.widgets import *

import os.path
import warnings
warnings.filterwarnings('ignore')

import gdown

# The Selfie Classifier!

Here you can do a very nice and fancy thing - you can detect if the given headshot is selfie or not.

Important:
  This only handles adult human headshots (man or woman). It might not give you a sensible answser for kids or full pose shots of the people.

Enjoy !

----

In [ ]:
class SelfieDetectorApp():
    MODEL_ID_FILE = 'model_id.txt'

    def __init__(self):
        model_path = self.download_model()
        if model_path and len(model_path) > 0:
          self.learner = load_learner(model_path, cpu=True)
        else:
          raise ValueError("Unable to load a model. Please check if the export.pkl is available for a given Google Drive File Id")
        self.upload = widgets.FileUpload()
        self.img_out = widgets.Output()
        self.result = widgets.Label()
        self.upload.observe(self._on_data_change, names=['data'])

    def download_model(self):
      if not os.path.isfile(self.MODEL_ID_FILE) or os.path.getsize(self.MODEL_ID_FILE) <= 1:
        print('Missing "model_id.txt" file or file is empty. Please add the file with the Google Drive File ID of the model')
        return None  

      model_id = open(self.MODEL_ID_FILE).read().split('\n')[0]
      return gdown.download(f'https://drive.google.com/uc?id={model_id}', 'export.pkl', quiet=False)

    def _on_data_change(self, change):
        self.result.value = ''
        img = PILImage.create(self.upload.data[-1])
        self.img_out.clear_output()
        with self.img_out: display(img.to_thumb(300,300)) 
        class_name, prob_idx, probs = self.learner.predict(img)
        self.result.value = f"{class_name.upper()} with {probs[prob_idx.item()]*100:.2f}% probability"

    def start(self):
        display(widgets.VBox([widgets.Label('Upload the headshot'), self.upload, self.img_out, self.result]))

app = SelfieDetectorApp()
app.start()